In [7]:
import geopandas as gpd
import networkx as nx
import folium

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Iterate through the GeoDataFrame
for index, row in gdf.iterrows():
    if 'power' in row and row['power'] == 'tower':
        point = row['geometry']
        coordinates = (point.y, point.x)  # Swap latitude and longitude
        node_id = row['id']
        G.add_node(node_id, pos=coordinates)  # Store coordinates as node attribute

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add markers for power tower nodes
for node_id, attrs in G.nodes(data=True):
    coords = attrs['pos']
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Save the map as an HTML file
m.save('power_tower_network.html')

# Display the map in the notebook
m

In [5]:
import geopandas as gpd
import networkx as nx
import folium

from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges to the graph
for i, j, w in zip(mst.row, mst.col, mst.data):
    G.add_edge(tower_nodes[i][0], tower_nodes[j][0], weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Update the folium map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
m.save('power_tower_network_with_links.html')


KeyError: 'pos'

In [3]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Add nodes to the graph with 'pos' attribute
for node_id, coords in tower_nodes:
    G.add_node(node_id, pos=coords)

# Verify all nodes have 'pos' attribute
for node_id, attrs in G.nodes(data=True):
    if 'pos' not in attrs:
        print(f"Node {node_id} is missing 'pos' attribute!")

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges from MST to the graph and verify nodes exist
for i, j, w in zip(mst.row, mst.col, mst.data):
    node_i, node_j = tower_nodes[i][0], tower_nodes[j][0]
    
    if node_i not in G:
        print(f"Node {node_i} not in graph!")
    if node_j not in G:
        print(f"Node {node_j} not in graph!")
    
    G.add_edge(node_i, node_j, weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Add edges to the map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

# Add new nodes to the map
for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
m.save('power_tower_network_with_links.html')


In [4]:
import json
import pandas as pd

# Load the existing GeoJSON file
with open('export.geojson', 'r') as file:
    geojson = json.load(file)

# Read the CSV file
df = pd.read_csv('ManualPowerPoles.csv', header=None)

# Convert CSV data to a list of coordinates
new_coords = [list(map(float, coord.split(','))) for coord in df[0]]

# Calculate the next available node id based on the last one
last_node_id = int(geojson['features'][-1]['properties']['@id'].split('/')[-1])
next_node_id = last_node_id + 1

# Add new nodes to the GeoJSON structure
for coord in new_coords:
    feature = {
        "type": "Feature",
        "properties": {
            "@id": f"node/{next_node_id}",
            "power": "tower"
        },
        "geometry": {
            "type": "Point",
            "coordinates": [coord[1], coord[0]]  # Note the order: [longitude, latitude]
        },
        "id": f"node/{next_node_id}"
    }
    
    geojson['features'].append(feature)
    next_node_id += 1

# Save the updated GeoJSON back to the file
with open('updated_export.geojson', 'w') as file:
    json.dump(geojson, file)


In [5]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Add nodes to the graph with 'pos' attribute
for node_id, coords in tower_nodes:
    G.add_node(node_id, pos=coords)

# Verify all nodes have 'pos' attribute
for node_id, attrs in G.nodes(data=True):
    if 'pos' not in attrs:
        print(f"Node {node_id} is missing 'pos' attribute!")

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges from MST to the graph and verify nodes exist
for i, j, w in zip(mst.row, mst.col, mst.data):
    node_i, node_j = tower_nodes[i][0], tower_nodes[j][0]
    
    if node_i not in G:
        print(f"Node {node_i} not in graph!")
    if node_j not in G:
        print(f"Node {node_j} not in graph!")
    
    G.add_edge(node_i, node_j, weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Add edges to the map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

# Add new nodes to the map
for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
m.save('power_tower_network_with_links_updated.html')


In [6]:
import geopandas as gpd
import networkx as nx
import folium
import numpy as np
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Function to connect using k-nearest neighbors
def knn_connections(dist_matrix, k=2):
    edges = []
    for i, distances in enumerate(dist_matrix):
        nearest_indices = np.argsort(distances)[1:k+1]  # Exclude the first one since it's the node itself
        for j in nearest_indices:
            edges.append((i, j))
    return edges

# Function to connect based on distance threshold
def threshold_connections(dist_matrix, threshold=1.5):
    edges = []
    for i in range(len(dist_matrix)):
        for j in range(i+1, len(dist_matrix)):
            if dist_matrix[i][j] < threshold:
                edges.append((i, j))
    return edges

# Function to randomly connect nodes
def random_connections(num_nodes, num_edges=10):
    edges = set()
    while len(edges) < num_edges:
        i, j = np.random.choice(num_nodes, 2, replace=False)
        if i > j:
            i, j = j, i
        edges.add((i, j))
    return list(edges)

# List of distinct colors for the different configurations
colors = ['blue', 'green', 'red', 'purple']

configurations = [
    ("Minimum Spanning Tree", [(i, j) for i, j, _ in zip(mst.row, mst.col, mst.data)]),
    ("K-Nearest Neighbors", knn_connections(dist_matrix, k=2)),
    ("Threshold-based Connection", threshold_connections(dist_matrix, threshold=1.5)),
    ("Random Connections", random_connections(len(positions), num_edges=10))
]

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

for color, (config_name, edges) in zip(colors, configurations):
    for i, j in edges:
        locs = [tower_nodes[i][1], tower_nodes[j][1]]
        folium.PolyLine(locs, color=color, weight=2.5, opacity=1, tooltip=config_name).add_to(m)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Add new nodes to the map
for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
#m.save('power_tower_network_with_links_updated.html')
m